## Modules

In [1]:
import numpy as np
import pandas as pd
import os, cv2, csv, random, sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10

## Preprocessing

In [2]:
data_path = '../input/ck/CK+48'
data_dir_list = os.listdir(data_path)

img_rows=256
img_cols=256
num_channel=1

num_epoch=10

img_data_list=[]
major_data_list= []
buffer_data_list = []
proper_data_list=[]

emotion_int = 1
image_count = 0
emotions_dict = {}

# processing all emotions
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img_resize)
    major_data_list.append(img_data_list)
    img_data_list = []

# adding emotion codes (1,2,3,..) to image sets
for emotion_sub in major_data_list:
    for sublist in emotion_sub:
        for img in sublist:
            for pixl in img:
                for intgr in pixl:
                    buffer_data_list.append(intgr.tolist())
        image_count += 1
        proper_data_list.append([emotion_int])
        proper_data_list.append(buffer_data_list)
        buffer_data_list = []
    emotions_dict[emotion_int] = image_count
    image_count = 0
    emotion_int += 1

print('Done!')

Loaded the images of dataset-anger

Loaded the images of dataset-contempt

Loaded the images of dataset-disgust

Loaded the images of dataset-fear

Loaded the images of dataset-happy

Loaded the images of dataset-sadness

Loaded the images of dataset-surprise

Done!


## Export to csv

In [3]:
f = open('tempCsvFile.csv','w', newline='')
writer = csv.writer(f)
k = 0
emotion_int_list = [1, 2, 3, 4, 5, 6, 7]
print('Writing temp csv file...')

for emotion_dict, emotion_count in emotions_dict.items():
    n = 0

    training_limit = round(emotion_count * 70 /100)
    public_T_limit = round(emotion_count * 85 /100)
    
    d_list = proper_data_list[k*2:(emotion_count + k)*2]
    
    # split dataset into ~70% training data and ~30% test data

    while n < (len(d_list) - 1):
        added_values = []
        label_value = ''
        #pixel_value = ' '.join(proper_data_list[k+1])
        pixel_value = ' '.join([str(elem) for elem in d_list[n+1]]) 
        added_values.append(pixel_value)
        if n <= (training_limit*2):
            label_value = 'Training'
        elif n <= (public_T_limit*2):
            label_value = 'PublicTest'
        else:
            label_value = 'PrivateTest'
        added_values.append(label_value)
        writer.writerow(d_list[n] + added_values)
        n += 2
    k += emotion_count
    
## Python will convert \n to os.linesep
f.close()
print('Done!')

Writing temp csv file...
Done!


## Read csv, then randomly shuffle rows

In [4]:
df = pd.read_csv('tempCsvFile.csv', header=0,dtype=object,na_filter=False)

print('Writing final csv file...')

df.columns =['emotion', 'pixels', 'Usage'] 
df_train = df.loc[df['Usage'] == 'Training']
df_publicT = df.loc[df['Usage'] == 'PublicTest']
df_privateT = df.loc[df['Usage'] == 'PrivateTest']

shuffled_train = df_train.reindex(np.random.permutation(df_train.index))
shuffled_publicT = df_publicT.reindex(np.random.permutation(df_publicT.index))
shuffled_privateT = df_privateT.reindex(np.random.permutation(df_privateT.index))

frames = [shuffled_train, shuffled_publicT, shuffled_privateT]
shuffled = pd.concat(frames)

shuffled.to_csv('ck+.csv', sep=',',encoding = 'utf-8',index = False)
os.remove("tempCsvFile.csv")

print('Done!')

Writing final csv file...
Done!
